#### 2019/09/14 Notes
-
* 第三部分的實作需要<span style="color:red">**價量資料**</span>
<br>
成交量/成交量週轉率 

<img src="https://i.imgur.com/YZ8f21o.jpg" height="300" width="300" >
<img src="https://i.imgur.com/VRp7NCJ.jpg" height="300" width="300">

<img src="https://i.imgur.com/1giFKNu.png" height="300" width="300">

### 第一部分：
- 基本統計量說明: Raw Data在當沖前/當沖後的各種基本統計量陳述（平均數、中位數、min、max、標準差），對Raw Data有個基本的概念

### 第二部分：
- 當沖前/當沖後的各種基本統計量的檢定（ex:現股當沖比重、日報酬率、週報酬率、月報酬率）
- 做平均數檢定、中位數檢定，看看當沖前/當沖後這些基本統計量有沒有顯著變化

### 第三部分：
- 政策面的研究。政府開放當沖是為了縮小spread、提高成交量、提高成交量週轉率。
- 研究data做的回歸是不是符合政府所宣稱的?當沖真的有穩定市場嗎？對資本市場有貢獻嗎？

### 第四部分：
- 市場面研究。有沒有其他在市場上面的因素會影響我們的回歸式？要控制這些變數，放入回歸式（很多x)

### $ \Delta $ $上市公司日報酬率標準差_i$ = $a_0$ + $a_1$ * (上市公司現股當沖比重平均)

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import statsmodels.stats.api as sms
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings
import datetime as datetime
import json
import os
import csv
from datetime import timedelta
from openpyxl import load_workbook
warnings.filterwarnings("ignore")
%matplotlib inline

from sklearn.linear_model import LinearRegression
from scipy import stats

In [87]:
class tw_day_trade():
    def __init__(self):
        self.pos_1 = ""
        self.pos_2 = ""
        self.pos_3 = ""
        self.df_before = "" 
        self.df_after = ""  
        self.basic_stock_stats_info_dict = {}
        self.x = "" 
        self.y_daily = "" 
        self.y_weekly = "" 
        self.y_monthly = "" 
        self.day_trade_data = ""

        """
        當沖相關重要日期們
        
        第一階段：2014/1/6 開放單向當沖200檔
        「台灣50、台灣中型100、富櫃50」200檔。
        
        第二階段：2014/6/30 開放雙向當沖
        仍為「台灣50、台灣中型100、富櫃50」200檔。
        
        第三階段：2015/6/1 新增「得發行認購(售)權證標的」及「ETF」亦得為當沖標的
        此時有377檔。
        
        第四階段：2016/2/1 開放「所有可做融資融券」的股票
        上市櫃共1432檔。
        
        第五階段：2017/4/28 調降當沖稅率
        
        
        """
        self.stg_one = "2014/1/6"
        self.stg_two = "2014/6/30"
        self.stg_three = "2015/6/1"
        self.stg_four = "2016/2/1"
        self.stg_five = "2017/4/28"
        

    def read_raw_csv(self, str_filename):
        """
        讀進我們需要的raw data
        """
        self.day_trade_data = pd.read_csv(str_filename)
        print("Read Data Successfully")
        self.company_num = len(self.day_trade_data)/4
        print("data公司總數: {} 間".format(self.company_num))
        #print("data shape: {}".format(self.day_trade_data.shape))

        
    def day_trade_split(self, date_1, date_2, date_3):
        """
        根據date_1, date_2, date_3來切割dataframe
        """
        tmp_arr = np.array(list(self.day_trade_data))
        self.pos_1 = np.where(tmp_arr==date_1)[0][0]
        self.pos_2 = np.where(tmp_arr==date_2)[0][0]
        self.pos_3 = np.where(tmp_arr==date_3)[0][0]
        #print("date_1 pos: {}\ndate_2 pos: {}\ndate_3 pos: {}".format(self.pos_1,self.pos_2,self.pos_3))
        
        self.df_before = self.day_trade_data.iloc[:,self.pos_1:self.pos_2]
        self.df_after = self.day_trade_data.iloc[:,self.pos_2:(self.pos_3+1)]
        #print("df_before shape: {}".format(self.df_before.shape))
        #print("df after shape: {}".format(self.df_after.shape))
        
        
    def day_trade_stats(self):
        """
        第一部分：計算當沖前/當沖後的基本統計量，對Raw Data有個基本概念
        """
        
        # 平均數 (axis=1橫條往右apply func., axis=0是直條往下apply func.)
        # 取axis=1, 每間公司在一段期間內為一個單位
        self.df_before['mean_before'] = self.df_before.mean(axis=1)
        self.df_after['mean_after'] = self.df_after.mean(axis=1)
        # assign value to basic stock info dictionary
        self.basic_stock_stats_info_dict['mean_before_daily'] = self.df_before['mean_before'][0::4].mean()
        self.basic_stock_stats_info_dict['mean_after_daily'] = self.df_after['mean_after'][0::4].mean()
        self.basic_stock_stats_info_dict['mean_before_weekly'] = self.df_before['mean_before'][1::4].mean()
        self.basic_stock_stats_info_dict['mean_after_weekly'] = self.df_after['mean_after'][1::4].mean()
        self.basic_stock_stats_info_dict['mean_before_monthly'] = self.df_before['mean_before'][2::4].mean()
        self.basic_stock_stats_info_dict['mean_after_monthly'] = self.df_after['mean_after'][2::4].mean()
        # mean_before_cash: df_before的現股當沖比重
        # mean_after_cash: df_after的現股當沖比重
        self.basic_stock_stats_info_dict['mean_before_cash'] = self.df_before['mean_before'][3::4].mean()
        self.basic_stock_stats_info_dict['mean_after_cash'] = self.df_after['mean_after'][3::4].mean()
        
        # 標準差 
        self.df_before['std_before'] = self.df_before.std(axis=1)
        self.df_after['std_after'] = self.df_after.std(axis=1)
        # assign value to basic stock info dictionary
        self.basic_stock_stats_info_dict['std_before_daily'] = self.df_before['std_before'][0::4].mean()
        self.basic_stock_stats_info_dict['std_after_daily'] = self.df_after['std_after'][0::4].mean()
        self.basic_stock_stats_info_dict['std_before_weekly'] = self.df_before['std_before'][1::4].mean()
        self.basic_stock_stats_info_dict['std_after_weekly'] = self.df_after['std_after'][1::4].mean()
        self.basic_stock_stats_info_dict['std_before_monthly'] = self.df_before['std_before'][2::4].mean()
        self.basic_stock_stats_info_dict['std_after_monthly'] = self.df_after['std_after'][2::4].mean()
        self.basic_stock_stats_info_dict['std_before_cash'] = self.df_before['std_before'][3::4].mean()
        self.basic_stock_stats_info_dict['std_after_cash'] = self.df_after['std_after'][3::4].mean()
        
        # 中位數
        self.df_before['median_before'] = self.df_before.median(axis=1)
        self.df_after['median_after'] = self.df_after.median(axis=1)
        # assign value to basic stock info dictionary
        self.basic_stock_stats_info_dict['median_before_daily'] = self.df_before['median_before'][0::4].mean()
        self.basic_stock_stats_info_dict['median_after_daily'] = self.df_after['median_after'][0::4].mean()
        self.basic_stock_stats_info_dict['median_before_weekly'] = self.df_before['median_before'][1::4].mean()
        self.basic_stock_stats_info_dict['median_after_weekly'] = self.df_after['median_after'][1::4].mean()
        self.basic_stock_stats_info_dict['median_before_monthly'] = self.df_before['median_before'][2::4].mean()
        self.basic_stock_stats_info_dict['median_after_monthly'] = self.df_after['median_after'][2::4].mean()
        self.basic_stock_stats_info_dict['median_before_cash'] = self.df_before['median_before'][3::4].mean()
        self.basic_stock_stats_info_dict['median_after_cash'] = self.df_after['median_after'][3::4].mean()
        
        # max
        self.basic_stock_stats_info_dict['max_before_daily'] = self.df_before['mean_before'][0::4].max()
        self.basic_stock_stats_info_dict['max_after_daily'] = self.df_after['mean_after'][0::4].max()
        self.basic_stock_stats_info_dict['max_before_weekly'] = self.df_before['mean_before'][1::4].max()
        self.basic_stock_stats_info_dict['max_after_weekly'] = self.df_after['mean_after'][1::4].max()
        self.basic_stock_stats_info_dict['max_before_monthly'] = self.df_before['mean_before'][2::4].max()
        self.basic_stock_stats_info_dict['max_after_monthly'] = self.df_after['mean_after'][2::4].max()
        self.basic_stock_stats_info_dict['max_before_cash'] = self.df_before['mean_before'][3::4].max()
        self.basic_stock_stats_info_dict['max_after_cash'] = self.df_after['mean_after'][3::4].max()
        
        # min
        self.basic_stock_stats_info_dict['min_before_daily'] = self.df_before['mean_before'][0::4].min()
        self.basic_stock_stats_info_dict['min_after_daily'] = self.df_after['mean_after'][0::4].min()
        self.basic_stock_stats_info_dict['min_before_weekly'] = self.df_before['mean_before'][1::4].min()
        self.basic_stock_stats_info_dict['min_after_weekly'] = self.df_after['mean_after'][1::4].min()
        self.basic_stock_stats_info_dict['max_before_monthly'] = self.df_before['mean_before'][2::4].min()
        self.basic_stock_stats_info_dict['min_after_monthly'] = self.df_after['mean_after'][2::4].min()
        self.basic_stock_stats_info_dict['min_before_cash'] = self.df_before['mean_before'][3::4].min()
        self.basic_stock_stats_info_dict['min_after_cash'] = self.df_after['mean_after'][3::4].min()
        
        # 第一四分位數
        self.basic_stock_stats_info_dict['0.25_before_daily'] = np.quantile(self.df_before['mean_before'][0::4],0.25)
        self.basic_stock_stats_info_dict['0.25_after_daily'] = np.quantile(self.df_after['mean_after'][0::4],0.25)
        self.basic_stock_stats_info_dict['0.25_before_weekly'] = np.quantile(self.df_before['mean_before'][1::4],0.25)
        self.basic_stock_stats_info_dict['0.25_after_weekly'] = np.quantile(self.df_after['mean_after'][1::4],0.25)
        self.basic_stock_stats_info_dict['0.25_before_monthly'] = np.quantile(self.df_before['mean_before'][2::4],0.25)
        self.basic_stock_stats_info_dict['0.25_after_monthly'] = np.quantile(self.df_after['mean_after'][2::4],0.25)
        self.basic_stock_stats_info_dict['0.25_before_cash'] = np.quantile(self.df_before['mean_before'][3::4],0.25)
        self.basic_stock_stats_info_dict['0.25_after_cash'] = np.quantile(self.df_after['mean_after'][3::4],0.25)
        
         # 第三四分位數
        self.basic_stock_stats_info_dict['0.75_before_daily'] = np.quantile(self.df_before['mean_before'][0::4],0.75)
        self.basic_stock_stats_info_dict['0.75_after_daily'] = np.quantile(self.df_after['mean_after'][0::4],0.75)
        self.basic_stock_stats_info_dict['0.75_before_weekly'] = np.quantile(self.df_before['mean_before'][1::4],0.75)
        self.basic_stock_stats_info_dict['0.75_after_weekly'] = np.quantile(self.df_after['mean_after'][1::4],0.75)
        self.basic_stock_stats_info_dict['0.75_before_monthly'] = np.quantile(self.df_before['mean_before'][2::4],0.75)
        self.basic_stock_stats_info_dict['0.75_after_monthly'] = np.quantile(self.df_after['mean_after'][2::4],0.75)
        self.basic_stock_stats_info_dict['0.75_before_cash'] = np.quantile(self.df_before['mean_before'][3::4],0.75)
        self.basic_stock_stats_info_dict['0.75_after_cash'] = np.quantile(self.df_after['mean_after'][3::4],0.75)
        
        
        """
        第二部分：Raw Data各種基本統計量檢定
        """
        
        # 日/週/月報酬率平均數檢定 
        # [t-statistics, p-value], 取p-value存入字典
        self.basic_stock_stats_info_dict['tttest_mean_daily'] = stats.ttest_ind(self.df_before['mean_before'][0::4],self.df_after['mean_after'][0::4])[1]
        self.basic_stock_stats_info_dict['tttest_mean_weekly'] = stats.ttest_ind(self.df_before['mean_before'][1::4],self.df_after['mean_after'][1::4])[1]
        self.basic_stock_stats_info_dict['ttest_mean_monthly'] = stats.ttest_ind(self.df_before['mean_before'][2::4],self.df_after['mean_after'][2::4])[1]
        
        # 日/週/月報酬率標準差檢定
        # [t-statistics, p-value], 取p-value存入字典
        self.basic_stock_stats_info_dict['ttest_std_daily'] = stats.ttest_ind(self.df_before['std_before'][0::4],self.df_after['std_after'][0::4])[1]
        self.basic_stock_stats_info_dict['ttest_std_weekly'] = stats.ttest_ind(self.df_before['std_before'][1::4],self.df_after['std_after'][1::4])[1]
        self.basic_stock_stats_info_dict['ttest_std_monthly'] = stats.ttest_ind(self.df_before['std_before'][2::4],self.df_after['std_after'][2::4])[1]       

        # 日/週/月報酬率中位數檢定
        # [t-statistics, p-value], 取p-value存入字典
        self.basic_stock_stats_info_dict['ttest_median_daily'] = stats.ttest_ind(self.df_before['median_before'][0::4],self.df_after['median_after'][0::4])[1]
        self.basic_stock_stats_info_dict['ttest_median_weekly'] = stats.ttest_ind(self.df_before['median_before'][1::4],self.df_after['median_after'][1::4])[1]
        self.basic_stock_stats_info_dict['ttest_median_monthly'] = stats.ttest_ind(self.df_before['median_before'][2::4],self.df_after['median_after'][2::4])[1]
                                                                                
        #print(json.dumps(self.basic_stock_stats_info_dict, indent=2))
    
    def read_write_reports_csv(self,output_csv_filename):
        """
        讀/寫基本統計量、檢定結果進出csv
        """
        w = csv.writer(open(output_csv_filename,"w"))
        key_chinese = ['']
        for key, val in self.basic_stock_stats_info_dict.items():
            if key[0:5] != 'ttest':
                val = round(val,4)
                val = str(val)+'%'
            
            if key[0:5] == 'ttest':
                if val <= 0.05: #p-value <= 0.05
                    val = 'significant'
                else: #p-value > 0.05
                    val = 'not significant'
            
            w.writerow([key,val])
            #print(key)
            
    
    def read_write_reports_excel(self):
        wb = load_workbook('20190923_testing.xlsx')
        print(wb.sheetnames)
        s1 = wb['單向當沖基本統計量']
        #print(s1['G8'].value)
        
        for col in s1.iter_cols(min_row=5,max_row=8,min_col=2,max_col=8):	
            for cell in col:
                print(cell.value)


    def day_trade_calculate(self):
        """
        準備OLS的x和y
        """
        self.x = self.df_after['mean_after'][3::4]-self.df_before['mean_before'][3::4]
        self.y_daily = self.df_after['std_after'][0::4]-self.df_before['std_before'][0::4]
        self.y_weekly = self.df_after['std_after'][1::4]-self.df_before['std_before'][1::4]
        self.y_monthly = self.df_after['std_after'][2::4]-self.df_before['std_before'][2::4]
    

    def day_trade_OLS(self, date_freq):
        """
        根據傳進的date_freq，計算OLS(日/週/月)
        date_freq's value: 'daily','weekly','monthly'
        """
        if date_freq=='daily':
            self.x = sm.add_constant(self.x)
            self.y_daily = list(self.y_daily)
            model_daily = sm.OLS(self.y_daily, self.x).fit()
            print(model_daily.summary())

        if date_freq=='weekly':
            self.x = sm.add_constant(self.x)
            self.y_weekly = list(self.y_weekly)
            model_weekly = sm.OLS(self.y_weekly, self.x).fit()
            print(model_weekly.summary())

        if date_freq=='monthly':
            self.x = sm.add_constant(self.x)
            self.y_monthly = list(self.y_monthly)
            model_monthly = sm.OLS(self.y_monthly, self.x).fit()
            print(model_monthly.summary())

In [88]:
data = tw_day_trade()
data.read_raw_csv("報酬率_pei/上市公司20160428_20180428.csv")
data.day_trade_split('2016/4/28','2017/4/28','2018/4/27')
data.day_trade_stats()
data.read_write_reports_csv("output/調降稅率前後一年(上市公司).csv")
data.read_write_reports_excel()

Read Data Successfully
data公司總數: 1078.0 間
['單向當沖基本統計量', '雙向當沖基本統計量(200間)_上市', '雙向當沖基本統計量(377間)_上市', '雙向當沖基本統計量(1432間)_上市', '稅率調降基本統計量(半年)_上市', '稅率調降基本統計量(一年)_上市', '檢定結果']
0.0
0.0578899684822219
0.303243908204471
0.845678922485964
0.0
1.641668826
3.451163797
6.62099227825542
0.0
-0.710045454545454
-3.42718461538461
-15.606293006993
0.0
-0.00536363636363636
0.000230244755244696
-0.635029545454545
0.0
0.0209203782133359
0.167906190288584
0.546582020087474
0.0
0.124636363636363
0.677795104895105
2.34844493006993
0.0
0.513043356643356
2.51945034965034
11.8254027972027
